In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
!pip install -q torch transformers faiss-cpu pillow tqdm numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 70.5 MB/s eta 0:00:00


In [ ]:
!pip install -q torch transformers faiss-cpu pandas requests pillow tqdm numpy


In [ ]:
import os
import pandas as pd
import requests
import pickle
import numpy as np

from tqdm import tqdm
from PIL import Image
from io import BytesIO

import torch
import faiss
from transformers import CLIPProcessor, CLIPModel


In [ ]:
CSV_PATH = "/content/photos_url.csv"
os.path.exists(CSV_PATH)


True

In [ ]:
df = pd.read_csv(CSV_PATH)
print("Total rows:", len(df))
df.head(), df.columns


Total rows: 25000


(                                     photo_image_url
 0  https://images.unsplash.com/uploads/1413387620...
 1  https://images.unsplash.com/reserve/jEs6K0y1Sb...
 2  https://images.unsplash.com/uploads/1412192004...
 3  https://images.unsplash.com/reserve/ijl3tATFRp...
 4  https://images.unsplash.com/reserve/6vaWXsQuSW...,
 Index(['photo_image_url'], dtype='object'))

In [ ]:
SAMPLE_SIZE = 1000

df_sampled = df.sample(
    n=min(SAMPLE_SIZE, len(df)),
    random_state=42
).reset_index(drop=True)

print("Sampled rows:", len(df_sampled))


Sampled rows: 1000


In [ ]:
IMAGE_DIR = "/content/images"
os.makedirs(IMAGE_DIR, exist_ok=True)

URL_COLUMN = "photo_image_url"

image_paths = []

for idx, url in tqdm(enumerate(df_sampled[URL_COLUMN]), total=len(df_sampled)):
    img_path = os.path.join(IMAGE_DIR, f"{idx}.jpg")

    try:
        r = requests.get(url, timeout=10)
        img = Image.open(BytesIO(r.content)).convert("RGB")
        img.save(img_path)
    except:
        # fallback image to keep indexing aligned
        Image.new("RGB", (224, 224), (0, 0, 0)).save(img_path)

    image_paths.append(img_path)

print(f"Images downloaded: {len(image_paths)}")


 71%|███████▏  | 714/1000 [06:50<03:21,  1.42it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (161087488 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 1000/1000 [09:43<00:00,  1.71it/s]

Images downloaded: 1000


In [ ]:
len(os.listdir(IMAGE_DIR))

1000

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
embeddings = []

for img_path in tqdm(image_paths, desc="Generating CLIP embeddings"):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        features = model.get_image_features(**inputs)

    features = features / features.norm(dim=-1, keepdim=True)
    embeddings.append(features.cpu().numpy())

embeddings = np.vstack(embeddings).astype("float32")

print("Embedding shape:", embeddings.shape)



Generating CLIP embeddings: 100%|██████████| 1000/1000 [06:36<00:00,  2.52it/s]

Embedding shape: (1000, 512)


In [ ]:
OUTPUT_DIR = "/content/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

np.save(f"{OUTPUT_DIR}/image_embeddings.npy", embeddings)

with open(f"{OUTPUT_DIR}/image_paths.pkl", "wb") as f:
    pickle.dump(image_paths, f)

print("Embeddings and paths saved")


Embeddings and paths saved


In [ ]:
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

faiss.write_index(index, f"{OUTPUT_DIR}/image_index.faiss")

print("FAISS index saved")


FAISS index saved


In [ ]:
from google.colab import files

files.download("/content/output/image_embeddings.npy")
files.download("/content/output/image_paths.pkl")
files.download("/content/output/image_index.faiss")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>